In [1]:
from langchain_ollama import ChatOllama
from ddgs import DDGS
from langchain_core.tools import tool, Tool
import ast
import operator
import time
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
import math

In [ ]:
#llm  = ChatOllama(model="deepseek-r1:8b")
llm  = ChatOllama(model="llama3.2:3b")

In [3]:
@tool
def calculator(expression: str) -> str:
    """Calculator for mathematical expressions. Input: mathematical expression like '2+2' or 'abs(-5)'"""
    try:
        # Safe expression evaluator using AST
        def safe_eval(node):
            if isinstance(node, ast.Constant):
                return node.value
            elif isinstance(node, ast.BinOp):
                left = safe_eval(node.left)
                right = safe_eval(node.right)
                return {
                    ast.Add: operator.add,
                    ast.Sub: operator.sub,
                    ast.Mult: operator.mul,
                    ast.Div: operator.truediv,
                    ast.Pow: operator.pow,
                    ast.Mod: operator.mod,
                }[type(node.op)](left, right)
            elif isinstance(node, ast.UnaryOp):
                operand = safe_eval(node.operand)
                return {
                    ast.UAdd: operator.pos,
                    ast.USub: operator.neg,
                }[type(node.op)](operand)
            elif isinstance(node, ast.Call):
                func_name = node.func.id
                args = [safe_eval(arg) for arg in node.args]
                
                # Safe math functions
                safe_functions = {
                    'abs': abs,
                    'round': round,
                    'sqrt': math.sqrt,
                    'sin': math.sin,
                    'cos': math.cos,
                    'tan': math.tan,
                    'log': math.log,
                    'log10': math.log10,
                    'exp': math.exp,
                    'floor': math.floor,
                    'ceil': math.ceil,
                }
                
                if func_name in safe_functions:
                    return safe_functions[func_name](*args)
                else:
                    raise ValueError(f"Function {func_name} not allowed")
            elif isinstance(node, ast.Name):
                # Allow mathematical constants
                constants = {
                    'pi': math.pi,
                    'e': math.e,
                }
                if node.id in constants:
                    return constants[node.id]
                else:
                    raise ValueError(f"Variable {node.id} not allowed")
            else:
                raise ValueError(f"Operation {type(node)} not supported")
        
        # Parse and evaluate
        parsed = ast.parse(expression, mode='eval')
        result = safe_eval(parsed.body)
        return str(result)
        
    except Exception as e:
        return f"Math error: {str(e)}"

In [4]:
x = calculator.invoke("abs(-25) + sqrt(144)")
print("x =", x)
x = calculator.invoke("(5 + 7)")
print("x =", x)
x = calculator.invoke("-7.5 * 4")
print("x =", x)

x = 37.0
x = 12
x = -30.0


In [5]:
@tool
def ddg_search(query: str) -> str:
    """Search DuckDuckGo for current information. Input: search query string"""
    try:
        # Retry logic to handle transient issues, know issue
        results = None
        RETRY_COUNT = 5
        for i in range(RETRY_COUNT):
            with DDGS() as ddgs:
                results = ddgs.text(query, max_results=3)
            
            if results is not None and len(results) > 0:
                break
            time.sleep(1)

        if not results:
            return "No search results found"
            
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'No title')}\n"
                f"   {result.get('body', 'No description')}\n"
                f"   URL: {result.get('href', 'No URL')}"
            )
        
        return "\n\n".join(formatted_results)
    except Exception as e:
        return f"Search error: {str(e)}"


In [6]:
x = ddg_search("latest AI news")
print(x)

C:\Users\91896\AppData\Local\Temp\ipykernel_28904\1797766533.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  x = ddg_search("latest AI news")


1. Artificial intelligence - BBC News
   1 day ago - All the latest content about Artificial intelligence from the BBC.
   URL: https://www.bbc.com/news/topics/ce1qrvleleqt

2. Artificial Intelligence - Latest AI News and Analysis
   The latest artificial intelligence news coverage focusing on the technology, tools and the companies building AI technology.
   URL: https://www.wsj.com/tech/ai

3. The latest AI news we announced in June
   2 weeks ago - Here are Google’s latestAI updates from June 2025
   URL: https://blog.google/technology/ai/google-ai-updates-june-2025/


In [7]:
tools = [
    Tool(
        name="calculator",
        func=calculator,
        description="Calculator for mathematical expressions. Input: mathematical expression like '2+2' or 'abs(-5)'"
    ),
    Tool(
        name="ddg_search",
        func=ddg_search,
        description="Search DuckDuckGo for current information. Input: search query string"
    )
]

In [8]:
# NOT IN USE; BUT WILL WORK
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools. 

{tools}

You are strictly instructed to use the tools wherever applicable, like for math questions always use 'calculator' tool and not try to find it by yourself, for current information use 'ddg_search' tool.                                    
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, must be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}""")


In [9]:
prompt = hub.pull("hwchase17/react") 
#prompt.template = "You are strictly instructed to use the tools wherever applicable, like for math questions always use 'calculator' tool and not try to find it by yourself, for current information use 'ddg_search' tool." + prompt.template

print("Prompt:", prompt)

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,
    return_intermediate_steps=True
)
def run_agent(question: str) -> str:
    """Run agent synchronously"""
    try:
        result = agent_executor.invoke({"input": question})
        print("Agent result:")
        print(result)
        print("+"*30)
        return result["output"]
    except Exception as e:
        return f"Error: {str(e)}"

Prompt: input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [10]:
questions = [
        "What is (5 + 7) ?",
        "Search for latest AI developments in 2024", 
        "Find recent news about LangChain framework",
        "Calculate -7.5 * 4"
    ]
    

for i, question in enumerate(questions, 1):
    print(f"\n{i}. {question}")
    print("-" * 30)

    answer = run_agent(question)
    print(f"Answer: {answer}")
    print("-" * 50)


1. What is (5 + 7) ?
------------------------------


> Entering new AgentExecutor chain...
<think>
Okay, let's tackle this problem step by step. The user asked me to calculate (5 + 7). Hmm, that seems pretty straightforward. 

I'm thinking about how I can solve this mathematically without needing any external tools. Adding those two numbers together is basic arithmetic, so the calculator tool should handle it easily. But just to be sure, maybe I should double-check because sometimes simple additions can slip through if not careful.

Alright, adding 5 and 7... that's a total of 12. That seems right. The user might be testing my ability to use basic tools or ensuring accuracy with simple calculations. 

I don't think there's any need for searching here since the answer is purely mathematical. The DuckDuckGo search would probably just give me more ways to calculate it, but that wouldn't help as much as confirming with a calculator.

So, I'll proceed with using the calculator tool and in